# Basic operations in tensorflow

references: [cs20si](http://web.stanford.edu/class/cs20si/lectures/slides_02.pdf) and [tensorflow](https://www.tensorflow.org)

In [1]:
import tensorflow as tf
from util import newlogname

** Graph **

A Graph contains a set of tf.Operation objects, which represent units of computation; and tf.Tensor objects, which represent the units of data that flow between operations.

A default Graph is always registered, and accessible by calling tf.get_default_graph. Another typical usage involves the tf.Graph.as_default context manager, which overrides the current default graph for the lifetime of the context.

In [2]:
my_graph = tf.Graph()
with my_graph.as_default():
    a = tf.constant(5.0)
    b = tf.constant(6.0)
    c = a * b

In [3]:
print(a)
print(b)
print(c)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("mul:0", shape=(), dtype=float32)


In [4]:
print(a.graph)
print(b.graph)
print(c.graph)
old = c.graph

**We can print the graph to see its contents**

In [5]:
my_graph.as_graph_def()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 5.0
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 6.0
      }
    }
  }
}
node {
  name: "mul"
  op: "Mul"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
versions {
  producer: 21
}

**Session**

A class for running TensorFlow operations.

A Session object encapsulates the environment 
in which Operation objects are executed, and Tensor objects are evaluated.

In [6]:
with tf.Session(graph=my_graph) as sess:
    a = a.eval()
    b = sess.run(b)
    c = sess.run(c)    
    print(a)
    print(b)
    print(c)

5.0
6.0
30.0


# Session vs InteractiveSession
You sometimes see InteractiveSession instead of Session
The only difference is an InteractiveSession makes itself the default

In [7]:
sess = tf.InteractiveSession()
a = tf.constant([[2,2],[-8.3,20]],name ="a")
b = tf.constant([[1,2,4],[-1.1,2.2,4.3]],name ="b")
c = tf.matmul(a,b,name ="c")
d = tf.constant([1,2,3,4,5,6,6,7,7,7,7,8,7,656,4])
e = tf.reduce_mean(d)
f = tf.reduce_max(d)
g = tf.reduce_sum(d)
print(a)
print(b)
print(c)

Tensor("a:0", shape=(2, 2), dtype=float32)
Tensor("b:0", shape=(2, 3), dtype=float32)
Tensor("c:0", shape=(2, 3), dtype=float32)


In [8]:
new = c.graph
print("new = ", new)
print("old = ", old)

new =  <tensorflow.python.framework.ops.Graph object at 0x7fb6627a6240>
old =  <tensorflow.python.framework.ops.Graph object at 0x7fb6627f1b38>


In [9]:
print("matrix multiplication  = {}\n".format(c.eval()))
print("mean  = ",e.eval())
print("max  = ",f.eval())
print("sum  = ",g.eval())
sess.close()

matrix multiplication  = [[ -0.20000005   8.39999962  16.60000038]
 [-30.29999924  27.39999962  52.79999924]]

mean  =  48
max  =  656
sum  =  730


In [10]:
my_graph = tf.Graph()
with my_graph.as_default():
    a = tf.constant(5.0,name="a")
    b = tf.constant(6.0,name="b")
    c = tf.multiply(a,b,name="c")
with tf.Session(graph=my_graph) as sess:
    log_path = newlogname()
    print("\n&&&&&&&&& For TensorBoard visualization type &&&&&&&&&&&")
    print("\ntensorboard  --logdir={}\n".format(log_path))
    writer = tf.summary.FileWriter(log_path, sess.graph)
    print(sess.run(a))


&&&&&&&&& For TensorBoard visualization type &&&&&&&&&&&

tensorboard  --logdir=./graphs/07-05-2017_21-42-18

5.0


In [11]:
# !tensorboard --logdir=./graphs/06-05-2017_20-03-44

# Tensors with values 
- tf.zeros : creates a tensor of shape and all elements will be zeros (when ran in session)
- tf.zeros_like : creates a tensor of shape and type (unless type is specified) as the input_tensor but all elements are zeros.
- tf.ones :  similar as above but with 1.
- tf.ones_like : similar as above but with 1.
- tf.fill : creates a tensor filled with a scalar value

In [12]:
graph = tf.Graph() 
with graph.as_default():
    zeros1 = tf.zeros([2, 3], tf.int32, name="zeros1") 
    input_tensor = tf.constant([[0, 1],[0, 1]], name="input")
    zeros2 = tf.zeros_like(input_tensor, name="zeros2")
    ones1 = tf.ones([2, 3], tf.int32, name="ones1") 
    ones2 = tf.ones_like(input_tensor, name="ones2")
    only69 = tf.fill([2, 3], 69,name="only69") 
with tf.Session(graph=graph) as sess:
    x1,x2,x3,x4,x5 = sess.run([zeros1,
                               zeros2,
                               ones1,
                               ones2,
                               only69])
    print(x1,"\n")
    print(x2,"\n")
    print(x3,"\n")
    print(x4,"\n")
    print(x5,"\n")            


[[0 0 0]
 [0 0 0]] 

[[0 0]
 [0 0]] 

[[1 1 1]
 [1 1 1]] 

[[1 1]
 [1 1]] 

[[69 69 69]
 [69 69 69]] 



# Variables

A variable maintains state in the graph across calls to run(). You add a variable to the graph by constructing an instance of the class Variable.

The Variable() constructor requires an initial value for the variable, which can be a Tensor of any type and shape. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.


**Why tf.constant but tf.Variable and not
tf.variable?**

tf.Variable is a class, but tf.constant is an op

We use **tf.global_variables_initializer()** to initialize all variables.

But we can also initialize only a subset of variables using **tf.variables_initializer**

Or initialaze a single variable using the method **initializer*

In [13]:
graph = tf.Graph() 
with graph.as_default():
    a = tf.Variable(2, name="scalar")
    b = tf.Variable([2, 3], name="vector")
    c = tf.Variable([[0, 1], [2, 3]], name="matrix")
    W = tf.Variable(tf.zeros([784,10]))
    init = tf.global_variables_initializer()
with tf.Session(graph=graph) as sess:
    sess.run(init)
    x1, x2, x3, x4 = sess.run([a,b,c,W])
    print(x1,"\n")
    print(x2,"\n")
    print(x3,"\n")
    print(x4,"\n")

2 

[2 3] 

[[0 1]
 [2 3]] 

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] 



In [14]:
graph = tf.Graph() 
with graph.as_default():
    W = tf.Variable(tf.truncated_normal([2, 2]))
    x = tf.constant([[0, 1],[-1, 0]],dtype = "float32")
    result = tf.matmul(W,x)
with tf.Session(graph=graph) as sess:
    sess.run(W.initializer)
    result = sess.run(result)
    print(result)

[[-0.24306829 -0.17629932]
 [ 1.72148156 -0.13534911]]


**Methods to assign values to a variable: assign(), assign_add() and assign_sub()**


this operation needs to be run to take effect

In [15]:
graph = tf.Graph() 
with graph.as_default():
    W = tf.Variable(10.0)
    assign_op = W.assign(2 * W)
with tf.Session(graph=graph) as sess:
    sess.run(W.initializer)
    print("original W = ", W.eval())
    sess.run(assign_op)
    print("first assign: W = ",W.eval())
    sess.run(assign_op)
    print("second assign: W = ",W.eval())
    sess.run(W.assign_add(100.)) 
    print("third assign: W = ",W.eval())
    sess.run(W.assign_sub(200.)) 
    print("fourth assign: W = ",W.eval())
with tf.Session(graph=graph) as sess:
    sess.run(W.initializer)
    print("original W = ", W.eval())


original W =  10.0
first assign: W =  20.0
second assign: W =  40.0
third assign: W =  140.0
fourth assign: W =  -60.0
original W =  10.0


** Each session maintains its own copy of
variable **

In [16]:
graph = tf.Graph() 
with graph.as_default():
    W = tf.Variable(10.)
print("First Session")
with tf.Session(graph=graph) as sess:
    sess.run(W.initializer)
    print("original W = ", W.eval())
    sess.run(W.assign_add(2.)) 
    print("first assign: W = ",W.eval())
print("\nSecond Session")
with tf.Session(graph=graph) as sess:
    sess.run(W.initializer)
    print("original W = ", W.eval())
    sess.run(W.assign_sub(2.)) 
    print("second assign: W = ",W.eval())

First Session
original W =  10.0
first assign: W =  12.0

Second Session
original W =  10.0
second assign: W =  8.0


# Placeholders

TensorFlow provides a placeholder operation that must be fed with data on execution. TensorFlow's feed mechanism lets you inject data into any Tensor in a computation graph. A python computation can thus feed data directly into the graph.

Supply feed data through the feed_dict argument to a run() or eval() call that initiates computation.

While you can replace any Tensor with feed data, including variables and constants, the best practice is to use a placeholder op node. A placeholder exists solely to serve as the target of feeds. It is not initialized and contains no data. A placeholder generates an error if it is executed without a feed, so you won't forget to feed it.


**Quirk:**
- shape=None means that tensor of any shape will be accepted as value for  placeholder.
- shape=None is easy to construct graphs, but nightmarish for debugging
-  shape=None also breaks all following  shape inference, which makes many ops not work because they expect certain rank


In [17]:
graph = tf.Graph() 
with graph.as_default():
    a = tf.placeholder(tf.float32, shape=[3])
    b = tf.constant([5, 5, 5], tf.float32)
    c = a + b 
with tf.Session(graph=graph) as sess:
    all_dict = {0:{a: [1, 2, 3]}, 1:{a: [1, 2, 3], b:[1, 2, 3]}}
    for i in [0,1]:
        feed_dict = all_dict[i]
        result = sess.run(c,feed_dict=feed_dict)
        print(result)

[ 6.  7.  8.]
[ 2.  4.  6.]


# Lazy loading

## Lazy loading = Defer creating/initializing an object until it is needed

As you can see in the examples [18] and [19], the result is the same. But in [18] the add node is already on the graph and in [19] the node add is added 5 times to the graph definition. **As a result your graph gets bloated, slow to load and expensive to pass around.**

In [18]:
graph1 = tf.Graph() 
with graph1.as_default():
    a = tf.constant(5.0)
    b = tf.constant(6.0)
    c = tf.add(a, b) # the node is created before executing the graph
with tf.Session(graph=graph1) as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(5):
        print(sess.run(c))

11.0
11.0
11.0
11.0
11.0


In [19]:
graph2 = tf.Graph() 
with graph2.as_default():
    a = tf.constant(5.0)
    b = tf.constant(6.0)
with tf.Session(graph=graph2) as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(5):
        print(sess.run(tf.add(a, b)))  # the node is created while executing the graph

11.0
11.0
11.0
11.0
11.0


In [20]:
graph1.as_graph_def()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 5.0
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 6.0
      }
    }
  }
}
node {
  name: "Add"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "init"
  op: "NoOp"
}
versions {
  producer: 21
}

In [21]:
graph2.as_graph_def()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 5.0
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 6.0
      }
    }
  }
}
node {
  name: "init"
  op: "NoOp"
}
node {
  name: "Add"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Add_1"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Add_2"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "Add_3"
  op: "A